In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 300})

In [ ]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 13))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)

plot_num = 1

default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 10,
    "n_clusters": 3,
    "min_samples": 20,
    "xi": 0.05,
    "min_cluster_size": 0.1,
}

datasets = [
    (
        noisy_circles,
        {
            "damping": 0.77,
            "preference": -240,
            "quantile": 0.2,
            "n_clusters": 2,
            "min_samples": 20,
            "xi": 0.25,
        },
    ),
    (noisy_moons, {"damping": 0.75, "preference": -220, "n_clusters": 2}),
    (
        varied,
        {
            "eps": 0.18,
            "n_neighbors": 2,
            "min_samples": 5,
            "xi": 0.035,
            "min_cluster_size": 0.2,
        },
    ),
    (
        aniso,
        {
            "eps": 0.15,
            "n_neighbors": 2,
            "min_samples": 20,
            "xi": 0.1,
            "min_cluster_size": 0.2,
        },
    ),
    (blobs, {}),
    (no_structure, {}),
]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params["n_neighbors"], include_self=False
    )
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params["n_clusters"])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params["n_clusters"], linkage="ward", connectivity=connectivity
    )
    spectral = cluster.SpectralClustering(
        n_clusters=params["n_clusters"],
        eigen_solver="arpack",
        affinity="nearest_neighbors",
    )
    dbscan = cluster.DBSCAN(eps=params["eps"])
    optics = cluster.OPTICS(
        min_samples=params["min_samples"],
        xi=params["xi"],
        min_cluster_size=params["min_cluster_size"],
    )
    affinity_propagation = cluster.AffinityPropagation(
        damping=params["damping"], preference=params["preference"], random_state=0
    )
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average",
        affinity="cityblock",
        n_clusters=params["n_clusters"],
        connectivity=connectivity,
    )
    birch = cluster.Birch(n_clusters=params["n_clusters"])
    gmm = mixture.GaussianMixture(
        n_components=params["n_clusters"], covariance_type="full"
    )

    clustering_algorithms = (
        ("MiniBatch\nKMeans", two_means),
        ("Affinity\nPropagation", affinity_propagation),
        ("MeanShift", ms),
        ("Spectral\nClustering", spectral),
        ("Ward", ward),
        ("Agglomerative\nClustering", average_linkage),
        ("DBSCAN", dbscan),
        ("OPTICS", optics),
        ("BIRCH", birch),
        ("Gaussian\nMixture", gmm),
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the "
                + "connectivity matrix is [0-9]{1,2}"
                + " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning,
            )
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding"
                + " may not work as expected.",
                category=UserWarning,
            )
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, "labels_"):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(
            list(
                islice(
                    cycle(
                        [
                            "#377eb8",
                            "#ff7f00",
                            "#4daf4a",
                            "#f781bf",
                            "#a65628",
                            "#984ea3",
                            "#999999",
                            "#e41a1c",
                            "#dede00",
                        ]
                    ),
                    int(max(y_pred) + 1),
                )
            )
        )
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(
            0.99,
            0.01,
            ("%.2fs" % (t1 - t0)).lstrip("0"),
            transform=plt.gca().transAxes,
            size=15,
            horizontalalignment="right",
        )
        plot_num += 1

plt.show()

In [ ]:
def sample_mixture(N, pi, mu1, sigma1, mu2, sigma2):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.random.multivariate_normal(mu1, sigma1, np.sum(z))
    res[np.where(z == 0)] = np.random.multivariate_normal(mu2, sigma2, N-np.sum(z))
    return z, res

def sample_two_classes(mu1, sigma1, mu2, sigma2, pi=0.5, N=200, Ntest=None):
    z, x = sample_mixture(N, pi, mu1, sigma1, mu2, sigma2)
    if Ntest is None:
        return z, x
    else:
        test_z, test_x = sample_mixture(Ntest, pi, mu1, sigma1, mu2, sigma2)
        return z, x, test_z, test_x

def plot_points(ax, x, z, mus=None, mu1=None, mu2=None, sigmas=None, points_alpha=1.0, colors=['r', 'b', 'g', 'magenta', 'grey', 'purple', 'darkgreen', 'orange', 'black']):
    print(set(z))
    for i in set(z):
        ax.scatter(x[np.where(z==i), 0], x[np.where(z==i), 1], marker='.', color=colors[i % 9], alpha=points_alpha)
        if sigmas is not None:
            plot_ellipse(ax, mus[i], sigmas[i], colors[i % 9])
    if mus is not None:
        for i in range(mus.shape[0]):
            ax.scatter([mus[i, 0]], [mus[i, 1]], marker='*', s=200, color=colors[i % 9])
    if mu1 is not None:
        for i in range(mu1.shape[0]):
            ax.scatter([mu1[i, 0]], [mu1[i, 1]], marker='*', s=200, color=colors[0])
    if mu2 is not None:
        for i in range(mu2.shape[0]):
            ax.scatter([mu2[i, 0]], [mu2[i, 1]], marker='*', s=200, color=colors[1])

from matplotlib import colors
cmap = colors.LinearSegmentedColormap(
    'red_blue_classes',
    {'red': [(0, 1, 1), (1, 0.7, 0.7)],
     'green': [(0, 0.7, 0.7), (1, 0.7, 0.7)],
     'blue': [(0, 0.7, 0.7), (1, 1, 1)]})

def plot_ellipse(ax, mu, sigma, color):
    v, w = sp.linalg.eigh(sigma)
    u = w[0] / sp.linalg.norm(w[0])
    angle = np.arctan(u[1] / u[0])
    angle = 180 * angle / np.pi
    ell = mpl.patches.Ellipse(mu, 2 * v[0] ** 0.5, 2 * v[1] ** 0.5,
                              180 + angle, facecolor=color,
                              edgecolor='black', linewidth=2)
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.2)
    ax.add_artist(ell)
    ax.scatter(mu[0], mu[1], marker='+', color=color, s=100)

def get_meshgrid(nx=200, ny=200):
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    return np.meshgrid(np.linspace(x_min, x_max, nx), np.linspace(y_min, y_max, ny))

def plot_colormesh(ax, model):
    xx, yy = get_meshgrid()
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap=cmap,
                   norm=colors.Normalize(0., 1.), zorder=0)
    plt.contour(xx, yy, Z, [0.5], linewidths=2., colors='white')

def plot_colormesh_svm(ax, model):
    xx, yy = get_meshgrid()
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=cmap,
                   norm=colors.Normalize(0., 1.), zorder=0)
    ax.contour(xx, yy, Z, [-1, 0, 1], linewidths=2., colors='white', linestyles=['--', '-', '--'])

def plot_svm_decision(ax, model, xx, yy):
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contour(xx, yy, Z, [0], linewidths=.5, colors='black', linestyles=['-'])

def plot_svm_decisions(ax, models):
    xx, yy = get_meshgrid()
    for model in models:
        plot_svm_decision(ax, model, xx, yy)
        
def sample_rings(rad_inner, rad_outer, rad_inner2, rad_outer2, pi=0.5, N=200, Ntest=None):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    rs = ( rad_inner + (rad_inner - rad_outer) * np.random.rand(np.sum(z)) )
    thetas = 2 * np.pi * np.random.rand(np.sum(z))
    rs2 = ( rad_inner2 + (rad_inner2 - rad_outer2) * np.random.rand(N - np.sum(z)) )
    thetas2 = 2 * np.pi * np.random.rand(N - np.sum(z))
    
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.array([ rs * np.cos(thetas), rs * np.sin(thetas) ]).T
    res[np.where(z == 0)] = np.array([ rs2 * np.cos(thetas2), rs2 * np.sin(thetas2) ]).T
    return z, res

def sample_mixture(N, pi, mu1, sigma1, mu2, sigma2):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.random.multivariate_normal(mu1, sigma1, np.sum(z))
    res[np.where(z == 0)] = np.random.multivariate_normal(mu2, sigma2, N-np.sum(z))
    return z, res

def sample_mixtures(mu0=np.array([ [-1,-1], [1,1] ]), sigma0=2, k=5, pi=0.5, df=4, N=200, Ntest=None):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    mus, sigmas = [], []
    for i, n in zip(range(2), [N-np.sum(z), np.sum(z)]):
        mus.append( np.random.multivariate_normal(mu0[i], sigma0 * np.identity(2), k) )
        sigmas.append( sp.stats.invwishart.rvs(df, np.identity(2), size=k) )
        mixture_ind = np.random.randint(k, size=n)
        cur_res = np.zeros((n, 2))
        for j in range(k):
            cur_indices = np.where(mixture_ind == j)[0]
            cur_res[cur_indices] = np.random.multivariate_normal(mus[-1][j], sigmas[-1][j], len(cur_indices))
        res[np.where(z == i)] = cur_res
    return z, res, mus, sigmas

def sample_oneclass_mixture(mu0=np.array([0, 0]), sigma0=2, k=5, pi=None, df=4, N=200, Ntest=None):
    mus = np.random.multivariate_normal(mu0, sigma0 * np.identity(2), k)
    sigmas = sp.stats.invwishart.rvs(df, np.identity(2), size=k)
    mixture_ind = np.random.randint(k, size=N)
    res = np.zeros((N, 2))
    for j in range(k):
        cur_indices = np.where(mixture_ind == j)[0]
        res[cur_indices] = np.random.multivariate_normal(mus[j], sigmas[j], len(cur_indices))
    return np.eye(k)[mixture_ind], res, mus, sigmas


## EM-алгоритм для кластеризации

In [ ]:
k = 4
z, x, true_mus, true_sigmas = sample_oneclass_mixture(k=k, sigma0=2, df=20, N=300)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
plot_points(ax, x, np.argmax(z, axis=1), mus=true_mus, sigmas=2 * true_sigmas, points_alpha=.5)
plt.show()

In [ ]:
def e_step(xs, pis, mus, sigmas):
    k = mus.shape[0]
    z = np.array([ pis[i] * sp.stats.multivariate_normal.pdf(xs, mean=mus[i], cov=sigmas[i]) for i in range(k) ])
    ## здесь z_{nk} = p(C_k)p(x_n|C_k)
    z = np.divide( z, np.sum(z, axis=0) ).T
    ## z_{nk} = p(C_k|x_n)
    return z

def m_step(xs, z):
    k = z.shape[1]
    pis = np.sum(z, axis=0) / np.sum(z)
    mus = np.array([np.average(xs, weights=z[:,i], axis=0) for i in range(k)])
    sigmas = np.array([np.cov(xs.T, aweights=z[:,i]) for i in range(k)])
    return pis, mus, sigmas

def loglikelihood(xs, pis, mus, sigmas):
    k = mus.shape[0]
    return np.sum(np.log(np.sum(np.array([ pis[i] * sp.stats.multivariate_normal.pdf(xs, mean=mus[i], cov=sigmas[i]) for i in range(k) ]), axis=0)))

In [ ]:
k = 4
mus = x[ np.random.choice(x.shape[0], size=k, replace=False), : ]
sigmas = np.array( [ np.identity(2) for _ in range(k) ] )
pis = (1./k) * np.ones(k)
z = e_step( x, pis, mus, sigmas)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
plot_points(ax, x, np.argmax(z, axis=1), mus=mus, sigmas=2 * sigmas, points_alpha=.5)
plt.show()

In [ ]:
pis, mus, sigmas = m_step(x, z)
z = e_step( x, pis, mus, sigmas)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
plot_points(ax, x, np.argmax(z, axis=1), mus=mus, sigmas=2 * sigmas, points_alpha=.5)
plt.show()

In [ ]:
pis

In [ ]:
k = 8
mus = x[ np.random.choice(x.shape[0], size=k, replace=False), : ]
sigmas = np.array( [ np.identity(2) for _ in range(k) ] )
pis = (1./k) * np.ones(k)
z = e_step( x, pis, mus, sigmas)

## EM-алгоритм
old_logl, new_logl = -np.inf, -np.inf
for iIter in range(5000):
    old_logl = new_logl
    z = e_step( x, pis, mus, sigmas)
    # print(z)
    new_pis, new_mus, new_sigmas = m_step(x, z)
    fig = plt.figure(figsize=(15,8))
    ax = fig.add_subplot(111)
    plot_points(ax, x, np.argmax(z, axis=1), mus=new_mus, sigmas=2 * new_sigmas, points_alpha=.5)
    plt.show()
    pis, mus, sigmas = new_pis, new_mus, new_sigmas
    new_logl = loglikelihood(x, pis, mus, sigmas)
    print("Логарифм правдоподобия на итерации %03d: %.6f" % (iIter, new_logl) )
    if np.abs(new_logl - old_logl) < 0.01:
        break

print("После %d итераций правдоподобие = %.6f" % (iIter, new_logl) )

In [ ]:
print(pis)
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
plot_points(ax, x, np.argmax(z, axis=1), mus=new_mus, sigmas=2 * new_sigmas, points_alpha=.5)
plt.show()

In [ ]:
likelihoods = {}

for k in range(2, 10):
    mus = x[ np.random.choice(x.shape[0], size=k, replace=False), : ]
    sigmas = np.array( [ np.identity(2) for _ in range(k) ] )
    pis = (1./k) * np.ones(k)

    ## EM-алгоритм
    old_logl, new_logl = -np.inf, -np.inf
    for iIter in range(50000):
        old_logl = new_logl
        z = e_step( x, pis, mus, sigmas)
        new_pis, new_mus, new_sigmas = m_step(x, z)
        pis, mus, sigmas = new_pis, new_mus, new_sigmas
        new_logl = loglikelihood(x, pis, mus, sigmas)
        if np.abs(new_logl - old_logl) < 0.00001:
            break
    print("k = %d\tПравдоподобие = %.6f" % (k, new_logl) )
    likelihoods[k] = new_logl

In [ ]:
def BIC(ll, N, d):
    return -ll + .5*np.log(N)*d

def AIC(ll, N, d):
    if N > d+1:
        return -ll + d + .5*(2*d*d+2*d) / (N - d - 1)
    else:
        return -ll + d + .5*(2*d*d+2*d)

In [ ]:
logLs = [ likelihoods[k] for k in sorted(likelihoods.keys()) ]
bics = [ BIC(likelihoods[k], x.shape[0], k-1 + k*( 2 + 3 ) ) for k in sorted(likelihoods.keys()) ]
aics = [ AIC(likelihoods[k], x.shape[0], k-1 + k*( 2 + 3 )  ) for k in sorted(likelihoods.keys()) ]

print("Логарифмы правдоподобий:\n%s" % logLs)
print("Значения AIC:\n%s" % aics)
print("Значения BIC:\n%s" % bics)
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
ax.plot(sorted(likelihoods.keys()), logLs, label="Log likelihood")
ax.plot(sorted(likelihoods.keys()), aics, label="AIC")
ax.plot(sorted(likelihoods.keys()), bics, label="BIC")
ax.legend(fontsize=legend_fontsize)
# ax.set_ylim((-320, 800))
plt.show()

## EM для кластеризации строк

In [ ]:
hamming_distance = lambda w1, w2: sum(s1 != s2 for s1, s2 in zip(w1, w2))
hamming_distance = np.vectorize(hamming_distance)

whole_wordset = Counter()
iLine = 0
with open('/media/snikolenko/2/datasets/wiki.processed') as f:
    for line in f:
        iLine += 1
        whole_wordset.update(line.split())
        if iLine == 10000:
            break

In [ ]:
m = 8
wordset = { k : v for k,v in whole_wordset.items() if len(k) == m }
all_words = np.array([k for k in wordset])

In [ ]:
all_words[:100]

In [ ]:
seed_words = np.random.choice([ w for w in all_words if whole_wordset[w] > 20], 20, replace=False)
print(sorted(seed_words))

In [ ]:
words = np.array([ w for w in all_words if np.min(hamming_distance(w, seed_words)) < 3 ])

In [ ]:
symbols = np.array([ a for a in set([ s for w in words for s in w ])])
smap = { v : i for (i, v) in enumerate(symbols) }
iwords = np.array([[smap[s] for s in w] for w in words])
ns = len(symbols)

In [ ]:
ns, sorted(seed_words), sorted(words)

In [ ]:
k = 8
num_samples = 2
sample_words = words[ np.random.choice(words.shape[0], size=num_samples*k, replace=False) ]
ps = (1./float(ns)) * np.ones(shape=(k, m, ns))
for i in range(k):
    for j in range(num_samples):
        for iS, s in enumerate(sample_words[num_samples*i + j]):
            ps[i, iS, smap[s]] += 1
ps_sums = np.sum(ps, axis=2)
for i in range(ps.shape[0]):
    for j in range(ps.shape[1]):
        ps[i, j, :] = ps[i, j, :] / ps_sums[i, j]
pis = (1./k) * np.ones(k)

In [ ]:
def e_step(xs, pis, logps):
    z = np.sum( np.array([ [logps[:, i, iwords[j][i]] for i in range(m)] for j in range(iwords.shape[0]) ]), axis = 1 ) + np.log(pis)
    z = z - np.logaddexp.reduce(z, axis=1).reshape(-1, 1)
    return z

def m_step(iwords, zs):
    z = np.exp(zs)
    pis = np.sum(z, axis=0) / np.sum(z)
    new_ps = np.ones(shape=(k, m, ns))
    for i in range(iwords.shape[0]):
        for ik in range(k):
            for im in range(m):
                new_ps[ik, im, iwords[i][im]] += z[i, ik]
    new_ps = np.divide( new_ps, np.sum( new_ps, axis = 2 ).reshape(k, m, 1) )
    return pis, new_ps

def print_clusters(z):
    zs = np.exp(z)
    for ik in range(k):
        print("Кластер %d:\n%s\n" % (ik, " ".join(np.sort([words[i] for i in range(z.shape[0]) if (zs[i, ik] > 0.7)]))))

In [ ]:
## один шаг EM-алгоритма; повторять по необходимости
z = e_step(iwords, pis, np.log(ps))
pis, ps = m_step(iwords, z)
print(pis)
print_clusters(z)

In [ ]:
for iIter in range(5):
    print("\n\n\t=== Итерация %d ===\n" % iIter)
    z = e_step(iwords, pis, np.log(ps))
    pis, ps = m_step(iwords, z)
    print(pis)
    print_clusters(z)